# Part 3

# Setup

In [1]:
using Plots
using JuMP, HiGHS

include("utils.jl")
include("extended_form.jl")
include("scenario_tree.jl")

scenario_tree (generic function with 1 method)

In [2]:
DATA_DIR = "data/"

"data/"

In [3]:
prices = vec(read_file(DATA_DIR * "belpex_price.txt"))
states = vec(read_file(DATA_DIR * "markov_support_4.txt"))
P_prob = read_file(DATA_DIR * "markov_weights_4.txt")

4×4 Matrix{Float64}:
 0.394106    0.524075   0.0807228  0.00109546
 0.107876    0.564185   0.311323   0.016616
 0.016616    0.311323   0.564185   0.107876
 0.00109546  0.0807228  0.524075   0.394106

# Questions

Now, consider the full multistge stochastic model

### 1. How many nodes are there in the full scenario tree ?

For N possible different states and T time steps, we have only one node at $t=1$ (assumed to be in state 1). At $t=2$, we then have $N$ possible outcomes (i.e children in the tree), whose transition probabilities are depending on the Markov chain, so at $t=2$, we have $N$ nodes. At $t=3$, all the possible nodes at $t=2$ also have $N$ children each, so $N^2$ nodes in total. We continue like this until $t=T$, where there are $N^{T-1}$ children nodes. In total, there are $\sum_{t=0}^{T-1} N^t$ nodes, which is equal to $\frac{N^T-1}{N-1}$ nodes. Setting $T=24$ and $N=4$, we then obtain $\frac{4^{24}-1}{3}$ nodes which is $\approx 9.38 \times 10^{13}$ nodes.

### 2. Implement both the extended formulation of the multistage stochastic program and the scenario tree formulation for $H < T$. Solve both formulations for $H=1,2,\dots$ until you reach the memory limit of your computer. What is the maximum $\bar{H}$ that your computer can handle ?

In [ ]:
N = 4
H = 11

# scenarios = generate_scenarios(H, N)
# num_scenarios = length(scenarios)

# λ_paths = Vector{Vector{Float64}}(undef, num_scenarios)
# probs = zeros(num_scenarios)
# for (i, s) in enumerate(scenarios)
#     λ_paths[i], probs[i] = compute_price_path(s,prices,states, P_prob)
# end

# extended_multistage(λ_paths, H, num_scenarios)

### 3. Plot the evolution of the number of variables and constraints for both formulations as H increases from $1$ to $\bar{H}$ 